In [2]:
import pandas as pd
csv_path = '../csv/BTC/KRW_BTC_day.csv'
df = pd.read_csv(csv_path)

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0', inplace=True)
df.columns = df.columns.str.capitalize()

In [3]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [6]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70

    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)

        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)

        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)


    def next(self):
        price = self.data.Close[-1]

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):

            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)

        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()
bt = Backtest(df, System, cash=1000000000, commission=.002)
stats = bt.run()
stats



Start                     2023-05-02 09:00:00
End                       2024-05-23 09:00:00
Duration                    387 days 00:00:00
Exposure Time [%]                   18.041237
Equity Final [$]                 1458258700.0
Equity Peak [$]                  1689799720.0
Return [%]                           45.82587
Buy & Hold Return [%]              147.103242
Return (Ann.) [%]                   42.601053
Volatility (Ann.) [%]               33.266453
Sharpe Ratio                         1.280601
Sortino Ratio                        3.603501
Calmar Ratio                         3.109049
Max. Drawdown [%]                  -13.702276
Avg. Drawdown [%]                   -3.730775
Max. Drawdown Duration       83 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      37.582233
Worst Trade [%]                     -7.582717
Avg. Trade [%]                    